# Mimic the Numpy ndarray in the Least Sense

In this notebook, I will try to write my version of ndarray providing the most common operations and behaviours of Numpy ndarray class, by which I believe I will obtain a tangible notion of the excellent space and time efficiency of Numpy ndarray.

In [4]:
class MyNdarray:
    def __init__(self, shape):
        self.shape = shape
        self.ndim = len(shape)
        import functools
        self.size = functools.reduce(lambda x, y: x*y, shape)
        self.data = [0 for i in range(self.size)]
        
    def __getlinearindex(self, key):
        index = 0
        acc = self.size
        for i in range(self.ndim):
            acc = acc / self.shape[i]
            index = index + key[i]*acc
        index = index - 1
        return index
        
    def __getitem__(self, key):
        if len(key) != self.size():
            raise ValueError("Index is illegal!")
        if any([key[i] < 0 or key[i] >= self.shape[i] for i in range(self.ndim)]):
            raise ValueError("Index is out of range!")
        index = self.__getlinearindex(key)
        return self.data[index]
    
    def __setitem__(self, key, value):
        if len(key) != self.size():
            raise ValueError("Index is illegal!")
        if any([key[i] < 0 or key[i] >= self.shape[i] for i in range(self.ndim)]):
            raise ValueError("Index is out of range!")
        if not isinstance(value, int):
            raise ValueError("Only support int value!")
        index = self.__getlinearindex(key)
        self.data[index] = value
        
    def __transform2string(self, dim, lo, unit, buffer):
        if dim == self.ndim - 1:
            buffer.write("[")
            for i in range(self.shape[dim]):
                buffer.write(str(self.data[lo+i]))
                if i < self.shape[dim] - 1:
                    buffer.write(", ")
            buffer.write("]")
            return 
        buffer.write("[")
        for i in range(self.shape[dim]):
            if i != 0:
                buffer.write(" "*(dim+1))
            self.__transform2string(dim+1, lo+unit*i, unit//self.shape[dim+1], buffer)
            if i < self.shape[dim] - 1:
                buffer.write(",\n")
        buffer.write("]")
            
        
    def __str__(self):
        import io
        buffer = io.StringIO()
        self.__transform2string(0, 0, self.size//self.shape[0], buffer);
        res = buffer.getvalue()
        buffer.close()
        return res
    
    @classmethod
    def __getlistnestlevel(cls, ls, acc):
        if isinstance(ls, list):
            if len(ls) == 0:
                raise ValueError("List should not be empty!")
            return cls.__getlistnestlevel(ls[0], acc+1)
        else:
            return acc
        
    @classmethod
    def __getlistshape(cls, ls, shape):
        if isinstance(ls, list):
            shape.append(len(ls))
            return cls.__getlistshape(ls[0], shape)
        else:
            return shape
    
    @classmethod
    def __assignlist2ndarray(cls, ls, arr, dim, lo, unit):
        if dim == arr.ndim - 1:
            for i in range(arr.shape[dim]):
                arr.data[lo+i] = ls[i]
        else:
            for i in range(arr.shape[dim]):
                cls.__assignlist2ndarray(ls[i], arr, dim+1, lo+unit*i, unit//arr.shape[dim+1])
    
    @classmethod
    def array(cls, ls):
        ndim = cls.__getlistnestlevel(ls, 0)
        shape = cls.__getlistshape(ls, [])
        res = MyNdarray(shape)
        cls.__assignlist2ndarray(ls, res, 0, 0, res.size//res.shape[0])
        return res
    
    def __add__(self, other):
        if other.shape != self.shape:
            raise ValueError("Addition require two operands are of the same shape.")
        res = MyNdarray(self.shape)
        for i in range(len(self.data)):
            res.data[i] = self.data[i] + other.data[i]
        return res
                    

In [5]:
arr = MyNdarray.array([[1, 2, 3], [2, 4, 6], [3, 6, 9]])
print(arr)

[[1, 2, 3],
 [2, 4, 6],
 [3, 6, 9]]


In [6]:
a = arr + arr
print(a)

[[2, 4, 6],
 [4, 8, 12],
 [6, 12, 18]]


## Basic test works, so in the following comes the more careful unit tests.